# Decision Tree from scratch, for classification problem, handling real and nominal values
**the used datasets are those ones, please feel free to download them first**

iris: https://archive.ics.uci.edu/ml/datasets/iris <br>
wine: https://archive.ics.uci.edu/ml/datasets/wine+quality <br>
house-votes-84: https://archive.ics.uci.edu/ml/datasets/congressional+voting+records <br>

In [ ]:
import numpy
import pandas
import plotly

In [75]:
def gini(p):
    return p * (1 - p)

def entropy(p):
    return -p * numpy.log2(p)

def compute_impurity(class_proportion, impurity_fct):
    stats = pandas.DataFrame(class_proportion["c"].value_counts())
    count = stats["c"].sum()
    tmp = stats.apply(lambda row: row["c"] / count, axis=1)
    stats.insert(1, "prob", tmp)
    
    segment_entropy = 0
    for prob in stats["prob"].values:
        segment_entropy += impurity_fct(prob)
    
    dominant_class = "error" if stats.empty else stats.idxmax()["c"]

    return count, segment_entropy, dominant_class, stats

def get_w_avg_feature(col, data, impurity_fct):
    # nominal
    col_list = data[col]
    if not pandas.api.types.is_numeric_dtype(col_list.dtype):
        total = data.shape[0]
        cat_list = col_list.unique()
        
        labels_ppt = []
        
        w_avg = 0
        for cat in cat_list:
            
            cat_data = data[data[col] == cat]
            c, s, i, stat = compute_impurity(cat_data, impurity_fct)
            # we keep segment entropy, the dominant class, category to compare with
            labels_ppt.append((s, i, cat, stat))
            w_avg += c / total * s
        return w_avg, labels_ppt, "nominal"
    
    # numerical
    else:
        # return array of segment from which we will build tree, impurity
        v = pandas.DataFrame([])
        col_list = col_list.sort_values().values
        col2 = col_list[1:]
        sr = pandas.Series(col2, index=range(0, len(col2)))
        v.insert(0, "col1", col_list)
        v.insert(1, "col2", sr)
        v = v[:-1]
        v.insert(2, "row-mean", col_list if len(col_list) == 1 else v.mean(axis=1))
        
        param_gt = None
        param_lt = None
        w_avg = None
        
        # once we have the mean we should classify data into two segment for each mean and see what its the better
        # think to optimise if you have already found a zero entropy, as it is a binary split
        
        for mean in v["row-mean"]:
            bin1 = data[mean >= col_list]
            bin2 = data[mean < col_list]

            c1, s1, i1, stat1 = compute_impurity(bin1, impurity_fct)
            c2, s2, i2, stat2 = compute_impurity(bin2, impurity_fct)

            total = c1 + c2
            # weighted average
            w_avg2 = (c1 / total) * s1 + (c2 / total) * s2
            if w_avg == None or w_avg > w_avg2:
                w_avg = w_avg2
                # we keep segment entropy, the dominant class, category to compare with
                param_gt = (s1, i1, mean, stat1)
                param_lt = (s2, i2, mean, stat2)
        if param_gt == None:
            print(v["row-mean"])
            print(col_list)
        return w_avg, [param_gt, param_lt], "numerical"

# this function help us to rank properties, and decide which is the next column that we will split
# I decide to split from the classes implementation because I first use to have the root, and the second time it is called localy by the sub trees
def wgh_avg_ranking(training_data, impurity_fct):
    # filter out target class
    cols = filter(lambda label: label != "c", training_data.columns.array)
    
    features_ppt = []
    for col in cols:
        # compute impurity weighted average
        ppt = get_w_avg_feature(col, training_data, impurity_fct)
        ppt = ppt + (col,)
        features_ppt.append(ppt)
        
    try:
        features_ppt.sort(key=lambda tpl: tpl[0])
    except:
        print(features_ppt)
    features_ppt.sort(key=lambda tpl: tpl[0])
    # return the feature with the lowest impurity
    return features_ppt[-1]

class Tree:
    # column name, value to compare with, column type, coltype numerical lt-gt/nominal, dominant class, impurity, data, impurity function (gini/entropy), stat, classes name
    def __init__(self, col, val, coltype, c, impurity, data, impurity_fct, stat, class_list, current_depth=0, max_depth=None, debug=False):
        self.col = col
        self.val = val
        self.coltype = coltype
        self.current_depth = current_depth + 1
        self.debug = debug
        if debug:
            symbol = "=="
            
            if coltype == "numerical-lt":
                symbol = "<"
            if coltype == "numerical-gt":
                symbol = ">"
            
            print(f"node:\ndepth = {current_depth}\ncondition = {col} {symbol} {val}")
        
        self.max_depth = max_depth
        
        # build occurence of classes object
        self.classStat = {}
        
        for i in class_list:
            self.classStat[i] = 0
            if i in stat.index:
                self.classStat[i] = stat["c"][i]
        
        self.class_list = class_list

        self.c = c
        self.impurity = impurity
        self.neighs = []
        self.impurity_fct = impurity_fct
        
        filtered_data = data[self.get_mask]

        #rest_features = data.columns.array#list(filter(lambda label: label != col, data.columns.array))
        
        rest_features = data.columns.array.tolist()
        if data[col].dtype == 'O':
            rest_features = list(filter(lambda label: label != col, rest_features))
        
        if impurity > 0 and len(rest_features) > 1 and not filtered_data.empty: # if only the target class remains
            if (max_depth != None and self.current_depth < max_depth) or max_depth == None:
                self.build_neighboor(filtered_data[rest_features], impurity_fct, class_list)

    def get_mask(self, data):
        if self.coltype == "numerical-lt":
            return data[self.col] < self.val
        if self.coltype == "numerical-gt":
            return data[self.col] >= self.val
        else:
            return data[self.col] == self.val

    def build_neighboor(self, data, impurity_fct, class_list):
        # calculate weighted average
        if data.empty:
            return []
        
        ft = wgh_avg_ranking(data, impurity_fct)
        
        roots = []
        _, splits, coltype, colname = ft
        
        if coltype == "numerical":
            (s1, i1, val1, stat1) = splits[0]
            roots.append(Tree(colname, val1, "numerical-gt", i1, s1, data, impurity_fct, stat1, class_list, self.current_depth, self.max_depth, self.debug))
            (s2, i2, val2, stat2) = splits[1]
            roots.append(Tree(colname, val2, "numerical-lt", i2, s2, data, impurity_fct, stat2, class_list, self.current_depth, self.max_depth, self.debug))
        else:
            for split in splits:
                (s, i, val, stat) = split
                roots.append(Tree(colname, val, coltype, i, s, data, impurity_fct, stat, class_list, self.current_depth, self.max_depth, self.debug))
        self.neighs = roots
    
    # get_stats allow us to retrieve probability for building the roc curve
    def test(self, data, get_stats=False):
        if self.neighs == [] or self.impurity == 0:
            filtered_data = data[self.get_mask(data)]
            filtered_data.insert(0, "pred", self.c)
            
            if get_stats:
                sr = pandas.DataFrame(self.classStat, index=filtered_data.index.values.tolist())
                filtered_data = pandas.concat([filtered_data, sr], axis=1)
                
            return filtered_data
        else:
            columns_list = list(data.columns) + ["pred"]
            if get_stats:
                columns_list = columns_list + self.class_list
            filtered_data = data[self.get_mask(data)]
            result = pandas.DataFrame(columns=columns_list)
            for tree in self.neighs:
                tmp = tree.test(filtered_data, get_stats)
                result = pandas.concat([result, tmp])
            return result

class DecisionTree:
    def __init__(self, training_data, target_class, method="Gini", max_depth=None, debug=False):
        impurity_fct = gini if method == "Gini" else entropy
        self.debug = debug
        
        # encountered some troubles with column name with dashes
        training_data = training_data.rename(columns=lambda s: s.replace("-", ""))
        self.roots = []
        self.max_depth = max_depth
        
        # make sur the target class is c
        training_data.rename(columns={target_class: "c"})
        
        self.class_list = training_data[target_class].unique().tolist()
        
        self.build(training_data, impurity_fct, self.class_list)
    
    def build(self, data, impurity_fct, class_list):
        feature_ppt = wgh_avg_ranking(data, impurity_fct)
        self.roots = self.process_sub_nodes(feature_ppt, data, impurity_fct, class_list)

    def process_sub_nodes(self, feature_ppt, data, impurity_fct, class_list):
        roots = []
        _, splits, coltype, colname = feature_ppt
        
        if coltype == "numerical":
            (s1, i1, val1, stat1) = splits[0]
            roots.append(Tree(colname, val1, "numerical-gt", i1, s1, data, impurity_fct, stat1, class_list, 0, self.max_depth, self.debug))
            (s2, i2, val2, stat2) = splits[1]
            roots.append(Tree(colname, val2, "numerical-lt", i2, s2, data, impurity_fct, stat2, class_list, 0, self.max_depth, self.debug))
        else:
            for split in splits:
                (s, i, val, stat) = split
                roots.append(Tree(colname, val, coltype, i, s, data, impurity_fct, stat, class_list, 0, self.max_depth, self.debug))
        return roots

    # get_stats allow us to retrieve probability for building the roc curve
    def test(self, data, get_stats=False):
        columns_list = (list(data.columns) + ["pred"])
        if get_stats:
            columns_list = columns_list + self.class_list
        
        result = pandas.DataFrame(columns=columns_list)
        data = data.rename(columns=lambda s: s.replace("-", ""))
        
        for root in self.roots:
            root_res = root.test(data, get_stats)
            result = pandas.concat([result, root_res])
        return result


In [49]:
# cross validation

def cross_validation(data, k=5, max_depth=None):
    # reindex row for cross validation
    data = data.sample(frac=1).reset_index(drop=True)    

    # issue with pandas query so we remove the dash from the classes's name beforehand
    data = data.rename(columns=lambda s: s.replace("-", ""))
    
    (line_nb, _) = data.shape
        
    sample_size = int(1/k * data.shape[0])
    kindexes = [i for i in range(0, line_nb, sample_size)]

    # for the roc curve
    column_names = ["c", "pred"] + data["c"].unique().tolist() # needs stats to have probability for the roc curve
    results = pandas.DataFrame(columns=column_names)

    for ki in kindexes:
        training_data = data.iloc[[(i < ki or i > (ki + sample_size)) for i in range(0, line_nb)]]
        test_data = data.iloc[[(i >= ki and (ki + sample_size) > i) for i in range(0, line_nb)]]

        dt = DecisionTree(training_data=training_data, target_class="c", method="Gini", max_depth=max_depth)
        result = dt.test(test_data, get_stats=True)
        
        results = pandas.concat([results, result[column_names]])
        n = result[result["c"] == result["pred"]].shape[0]
        print(f"accuracy = {n/len(result)}")
        total = result.shape[0]
    return results


**here is the roc curve implementation function, doesn't matter if the target class is multiclass, if so the roc curve will be go with pairwise analysis
this is the method description: https://stats.stackexchange.com/questions/105760/how-we-can-draw-an-roc-curve-for-decision-trees**

In [51]:


import plotly.graph_objects as go

def compare_threshold(col, threshold):
    if col > threshold:
        return 1
    return 0

def compute_rate(predictions, pairclass):
    
    p = predictions[predictions["c"] == pairclass[0]].shape[0]
    n = predictions[predictions["c"] == pairclass[1]].shape[0]

    tparr = predictions.apply(lambda row: row["c"] == pairclass[0] and row["pred"] == 1, axis=1)
    fparr = predictions.apply(lambda row: row["c"] == pairclass[1] and row["pred"] == 1, axis=1)
    
    count_arr = numpy.bincount(tparr)
    tp = 0
    if (count_arr.size != 1):
        tp = count_arr[1]
    
    count_arr = numpy.bincount(fparr)
    fp = 0
    if (count_arr.size != 1):
        fp = count_arr[1]

    tpr = tp / p if p != 0 else 0
    fpr = fp / n if n != 0 else 0
    
    return fpr, tpr

def get_intervals(fpr, tpr):
    indexes = [0]
    values = [fpr[0]]
    values2 = [tpr[0]]
    
    for i in range(0, len(fpr)):
        if not fpr[i] in values:
            indexes.append(i)
            values.append(fpr[i])
            values2.append(tpr[i])
        if fpr[i] in values and tpr[i] > values2[-1]:
            indexes[-1] = i
            values[-1] = fpr[i]
            values2[-1] = tpr[i]
            
    return indexes


def auc(tpr, fpr):
    # reverse list
    acc = 0
    tpr.reverse()
    fpr.reverse()
    
    indexes = get_intervals(fpr, tpr)

    for i in range(0, len(indexes)):
        if i == len(indexes) - 1:
            break
        i1 = indexes[i]
        i2 = indexes[i + 1]
        
        area = 0

        coef = (tpr[i2] - tpr[i1]) / (fpr[i2] - fpr[i1])
        
        if coef == 0:
            area = tpr[i1] * fpr[i2] - tpr[i1] * fpr[i1]
        else:
            y2 = coef * fpr[i1]
            b = tpr[i1] - y2
            
            area = (((fpr[i2]**2)/2 * coef) + b * fpr[i2]) - (((fpr[i1]**2)/2 * coef) + b * fpr[i1])
        acc += area
    return acc    

def roc_curve(predictions, pairclass):
    # not finished
    arr_fpr = []
    arr_tpr = []
    nbpoint = 500
    c1, c2 = pairclass    
    pred1 = predictions.apply(lambda row: (row[c1] / (row[c1] + row[c2])) if (row[c1] + row[c2]) != 0 else 0 , axis=1)
    predictions = predictions.drop(['pred'], axis=1)
    
    for t in range(0, nbpoint):
        pred = pred1.apply(lambda col: compare_threshold(col, t/nbpoint))
        predictions.insert(0, "pred", pred)
        fpr, tpr = compute_rate(predictions, pairclass)
        arr_fpr.append(fpr)
        arr_tpr.append(tpr)
        # recompute the prediction column based on the threshold
        predictions = predictions.drop(['pred'], axis=1)

    return (arr_tpr, arr_fpr)

def roc_curves(results):
    # build the class pair to compare with
    classes = list(results["c"].unique())
    classes2 = classes[1:].copy()
    pair_array = []
    for c in classes:
        if not len(classes2):
                break
        for c2 in classes2:
            pair_array.append((c, c2))
        classes2.pop(0)
    fig = go.Figure(layout_title_text="ROC curve")
    
    
    for pair in pair_array:
        # split concerning data
        (c1, c2) = pair
        
        cresults = results.query(f"c == '{c1}' | c == '{c2}'")
        
        # compute roc curve
        (tpr, fpr) = roc_curve(cresults, pair)
        

        # draw roc curve
        fig.add_trace(
            go.Scatter(
                x=fpr,
                y=tpr,
                name=f"AUC({c1}, {c2}) = {auc(tpr, fpr)})",
                mode="lines",
        ))
    fig.update_traces(textfont_size=14)
    fig.update_layout(legend_title_text='ROC Curve', showlegend=True)
    fig.show()
    
    #f"AUC({c1}, {c2}) = {auc(tpr, fpr)})"
    


In [80]:
#first 
data = pandas.read_csv("./house-votes-84.tmls", encoding="ascii", skiprows=[1]).rename(columns={"class": "c"})

# clean data/remove rows that have unknown value
data = data.drop(data[data["handicapped-infants"].str.contains(r"[?]") |
                        data["c"].str.contains(r"[?]") |
                        data["water-project-cost-sharing"].str.contains(r"[?]") |
                        data["adoption-of-the-budget-resolution"].str.contains(r"[?]") |
                        data["physician-fee-freeze"].str.contains(r"[?]") |
                        data["el-salvador-aid"].str.contains(r"[?]") |
                        data["religious-groups-in-schools"].str.contains(r"[?]") |
                        data["anti-satellite-test-ban"].str.contains(r"[?]") |
                        data["aid-to-nicaraguan-contras"].str.contains(r"[?]") |
                        data["mx-missile"].str.contains(r"[?]") |
                        data["immigration"].str.contains(r"[?]") |
                        data["synfuels-corporation-cutback"].str.contains(r"[?]") |
                        data["education-spending"].str.contains(r"[?]") |
                        data["superfund-right-to-sue"].str.contains(r"[?]") |
                        data["crime"].str.contains(r"[?]") |
                        data["duty-free-exports"].str.contains(r"[?]") |
                        data["export-administration-act-south-africa"].str.contains(r"[?]")
                        ].index)


# here i take out some column otherwise it would take time
#data = data[["c", "handicapped-infants", "water-project-cost-sharing", "duty-free-exports", "anti-satellite-test-ban"]]
results = cross_validation(data, k=8, max_depth=4)
roc_curves(results)

accuracy = 0.6551724137931034
accuracy = 0.6896551724137931
accuracy = 0.6428571428571429
accuracy = 0.6896551724137931
accuracy = 0.7586206896551724
accuracy = 0.6896551724137931
accuracy = 0.75
accuracy = 0.6428571428571429


In [8]:
pandas.read_csv("./iris.tmls", skiprows=[1]).rename(columns={"class": "c"})

,sepal length,sepal width,petal length,petal width,c
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [78]:
## second dataset
data = pandas.read_csv("./iris.tmls", skiprows=[1]).rename(columns={"class": "c"})
results = cross_validation(data, k=8, max_depth=1)
roc_curves(results)


accuracy = 0.3333333333333333
accuracy = 0.6111111111111112
accuracy = 0.3888888888888889
accuracy = 0.16666666666666666
accuracy = 0.3333333333333333
accuracy = 0.3333333333333333
accuracy = 0.16666666666666666
accuracy = 0.5555555555555556
accuracy = 0.3333333333333333


In [77]:
# third dataset
data = pandas.read_csv("./wine.tmls", skiprows=[1]).rename(columns={"class": "c"})
data["c"] = data["c"].astype(str) # for convenience
results = cross_validation(data, k=8, max_depth=1)
roc_curves(results)

accuracy = 0.5454545454545454
accuracy = 0.3181818181818182
accuracy = 0.4090909090909091
accuracy = 0.36363636363636365
accuracy = 0.36363636363636365
accuracy = 0.45454545454545453
accuracy = 0.4090909090909091
accuracy = 0.36363636363636365
accuracy = 0.5


# display nodes inside tree

In [72]:
data = pandas.read_csv("./wine.tmls", skiprows=[1]).rename(columns={"class": "c"})
model = DecisionTree(training_data=data[:130], target_class="c", method="Gini", max_depth=9, debug=True)

current depth = 1
node:
depth = 0
condition = hue > 1.04
current depth = 2
node:
depth = 1
condition = alcalinity of ash > 18.0
current depth = 3
node:
depth = 2
condition = magnesium > 88.0
current depth = 4
node:
depth = 3
condition = nonflavanoid phenols > 0.31
current depth = 4
node:
depth = 3
condition = nonflavanoid phenols < 0.31
current depth = 5
node:
depth = 4
condition = proline > 952.5
current depth = 5
node:
depth = 4
condition = proline < 952.5
current depth = 3
node:
depth = 2
condition = magnesium < 88.0
current depth = 2
node:
depth = 1
condition = alcalinity of ash < 18.0
current depth = 1
node:
depth = 0
condition = hue < 1.04
